In [1]:
import matplotlib.pyplot as plt
import numpy as np
import time
import pandas as pd

import pickle
import re
import glob
from sklearn import metrics
import os
import sys
import pymc3 as pm 
#Set some List
myAA = ["R","H","K","D","E","S","T","N","Q","C","G","P","A","V","I","L","M","F","Y","W"]
FullAmino = ["ARG","HIS","LYS","ASP","GLU","SER","THR","ASN","GLN","CYS","GLY","PRO","ALA","VAL","ILE","LEU","MET","PHE","TYR","TRP"]


In [2]:
#Load score from Etabx with some filtering coming from another file
AllScore = pickle.load(open("./data/AllNormScore.pk","rb"))

#Load what is the receptor contact AA identity
ReceptorContact = pickle.load(open("./data/ReceptorContact.pk","rb"))

In [3]:
Dataset = 9
Poly = 2
Rand = 0
RecepToRun = "MCL1"
Prior = "Struct"
Prior = "No"
Poly = 0
PrioStr = 1.0
RunName = "Test"

#[Dataset,Poly,Rand,RecepToRun,Prior,PrioStr,RunName] = sys.argv[1:]
Dataset = int(Dataset)
Poly = int(Poly)
Rand = int(Rand)

print("Dataset=%d" % (Dataset))
print("Poly=%d" % (Poly))
print("Random=%d" % (Rand))


Dataset=9
Poly=0
Random=0


In [4]:
model = "_".join([RunName,str(Dataset),RecepToRun,"rand",str(Rand),"poly",str(Poly),"Prior",Prior,"PrioStr","%.3f" % (PrioStr)])
#pickle.dump(trace,open("/scratch/users/vfrap/BayesianModel/"+model+"_small.pk","wb"))
print("Model=%s" % (model))
model_csv = "./results/"+model+"_cor.csv"
if os.path.isfile(model_csv):
    sys.exit()

Model=Test_9_MCL1_rand_0_poly_0_Prior_No_PrioStr_1.000


In [5]:
#Sort All Feature
AllFeature = sorted(list(AllScore.keys()))

if Poly == 0:
    Linear = []
    for s in AllFeature:
        sp = s.split(" ")
        if sp[0] == sp[1]:
            Linear.append(s)
    AllFeature = Linear
    
if Poly == 2:
    RecepPep = []
    for s in AllFeature:
        sp = s.split(" ")
        if sp[0].split(",")[-1] != sp[1].split(",")[-1]:
            RecepPep.append(s)
            #print(s)
        if sp[0] == sp[1]:
            RecepPep.append(s)
    AllFeature = RecepPep

print("This is the number of Feature=%d" % (len(AllFeature)))

#Build strc Prior
StrcMu = []
StrcSt = []
for s in AllFeature:
    StrcMu.append(np.mean(AllScore[s]))
    StrcSt.append(np.std(AllScore[s])+PrioStr)

This is the number of Feature=400


In [6]:
SmallRecep = "NA"
Test = []
for t in RecepToRun.split(","):
    if t == "B2CL1":SmallRecep = "x"
    if t == "MCL1":SmallRecep = "m"
    if t == "B2LA1":SmallRecep = "f"
    Test.append("mix_"+SmallRecep+"_ener")

In [7]:
BindingData = pd.read_csv("./cluster/clust_"+str(Dataset)+".csv")


In [8]:

#Encode Seq string on feature set
def encode_FeatDict(s,SortFeatInd,RecepCon,PepStart = 91,PepChain = "B"):
    vect = [0]*len(SortFeatInd)
    for i in range(len(s)):
        k1 = ",".join([s[i],str(i+PepStart),PepChain])
        #Do pep-pep
        for j in range(i,len(s)):
            k2 = ",".join([s[j],str(j+PepStart),PepChain])
            contK = " ".join(sorted([k1,k2]))
            if contK not in SortFeatInd:continue
            ind = SortFeatInd[contK]
            vect[ind] = 1
        for k2 in RecepCon:
            contK = " ".join(sorted([k1,k2]))
            if contK not in SortFeatInd:continue
            ind = SortFeatInd[contK]
            #print(contK,ind)
            vect[ind] = 1
    return(vect)


In [9]:
SortFeatInd = dict()
for i in range(len(AllFeature)):
    SortFeatInd[AllFeature[i]] = i

In [10]:
#Find where peptide start
minn = 999999
for s in AllFeature:
    sp = s.split(" ")
    if ",B" in sp[0]:
        num = int(sp[0].split(",")[1])
        if num < minn:
            minn = num
            print(s)
print(minn)

A,100,B A,100,B
A,91,B A,91,B
91


In [11]:
X = []
Y = []
for (t,r) in zip(Test,RecepToRun.split(",")):
    TrainDF = BindingData.dropna(subset=[t])
    print(t,r,len(TrainDF))
    for (seq,val) in zip(TrainDF["twenty"].values,TrainDF[t].values):
        enc = encode_FeatDict(seq,SortFeatInd,ReceptorContact[r],PepStart=minn)
        X.append(enc)
        Y.append(val)
X = np.array(X)
Y = np.array(Y)
print(len(Y))

mix_m_ener MCL1 182
182


In [12]:
df = pd.read_csv("./cluster/random.csv")

In [13]:
Y_rand = []
X_rand = []
for r in (["B2CL1","B2LA1","MCL1"]):
    for (s,v) in zip(list(df["EncodeAA"].values)[:int(len(df)/2)],list(df["Ener"].values)[:int(len(df)/2)]):
        enc = encode_FeatDict(seq,SortFeatInd,ReceptorContact[r],PepStart=minn)
        X_rand.append(enc)
        Y_rand.append(v)
X_rand = np.array(X_rand)
Y_rand = np.array(Y_rand)

In [14]:
basic_model = pm.Model()
#Prior = "No"
with basic_model:
    print("Define alphas")
    # Priors for unknown model parameters
    if Prior == "No":
        alphas = pm.Normal('alphas', mu=0.0, sd=PrioStr,shape=len(X[0]))
    if Prior == "Struct":
        print("Struct Prior")
        alphas = pm.Normal('alphas', mu=np.array(StrcMu), sd=np.array(StrcSt),shape=len(X[0]))
        #alphas = pm.Normal('alphas', mu=np.array(StrcMu), sd=PrioStr,shape=len(X[0]))
    print("Define beta")
    beta = pm.Normal('beta', mu=np.mean(Y), sd=1)
    
    print("Define Expected")
    # Expected value of outcome
    
    mscale = pm.Normal('scale', mu=2, sd=1)
    
    mu = beta + pm.math.dot(alphas, X.transpose())*mscale
    #mu = beta + pm.math.dot(alphas, X.transpose())
    
    #sigma = pm.HalfCauchy('sigma', beta=10, testval=1.)
    
    print("Define Likehood")
    # Likelihood (sampling distribution) of observations
    Y_obs = pm.Normal('Y_obs', mu=mu, sd=0.5, observed=Y)
    
    #Random data
    if Rand == 1:
        randomdata = pm.Normal('Y_rand', mu=beta + pm.math.dot(alphas, X_rand.transpose())*mscale, sd=2.0, observed=Y_rand)
    
    


# In[ ]:


with basic_model:
    print("Sampling")
    # Inference!
    #start = pm.find_MAP() # Find starting value by optimization
    #step = pm.NUTS(state=start) # Instantiate MCMC sampling algorithm
    #trace = pm.sample(2000, start=start)
    trace = pm.sample(tune=3000,draws=2000)
    #trace = pm.sample(2000)
print("Done!")

Define alphas
Define beta
Define Expected
Define Likehood


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...


Sampling


100%|█████████▉| 4999/5000 [02:14<00:00, 37.29it/s]/media/vince/anaconda/envs/py35/lib/python3.5/site-packages/pymc3/step_methods/hmc/nuts.py:467: UserWarning: Chain 0 contains 6 diverging samples after tuning. If increasing `target_accept` does not help try to reparameterize.
  % (self._chain_id, n_diverging))
100%|██████████| 5000/5000 [02:14<00:00, 37.29it/s]

Done!


In [15]:
TrainRecepToRun = RecepToRun
AllCors = dict()
c = 0
for RecepToRun in ["B2CL1","MCL1","B2LA1"]:
    for f in sorted(glob.glob("./cluster/clust_*.csv")):
        BindingData = pd.read_csv(f)
    
        SmallRecep = "NA"
        if RecepToRun == "B2CL1":SmallRecep = "x"
        if RecepToRun == "MCL1":SmallRecep = "m"
        if RecepToRun == "B2LA1":SmallRecep = "f"
        Test = "mix_"+SmallRecep+"_ener"
        TrainDF = BindingData.dropna(subset=[Test])
        X = []
        Y = []
        for (seq,val) in zip(TrainDF["twenty"].values,TrainDF[Test].values):
            enc = encode_FeatDict(seq,SortFeatInd,ReceptorContact[RecepToRun],PepStart=minn)
            X.append(enc)
            Y.append(val)
        X = np.array(X)
        Y = np.array(Y)
        AllCor = []
        for i in range(1000,2000):
            w = trace.get_values("alphas")[i]
            m = trace.get_values("scale")[i]
            b = trace.get_values("beta")[i]
            
            xi = np.dot(w, X.transpose())*m+b
            yi = Y
            
            cor = np.corrcoef(xi,yi)[0][1]
            AllCor.append(cor)
        c = c+1
        AllCors[c] = dict()
        AllCors[c]["File"] = f
        AllCors[c]["RecepToRun"] = RecepToRun
        AllCors[c]["MeanCor"] = np.mean(AllCor)
        AllCors[c]["StdCor"] = np.std(AllCor)
        AllCors[c]["Dataset"] = Dataset
        AllCors[c]["ReceptorSet"] = TrainRecepToRun
        AllCors[c]["Random"] = Rand
        AllCors[c]["Poly"] = Poly
        AllCors[c]["Model"] = model
        AllCors[c]["Prior"] = Prior
        AllCors[c]["PriorStr"] = PrioStr
        AllCors[c]["Test"] = RunName
        print(f,RecepToRun,"%.2f %.2f" % (np.mean(AllCor),np.std(AllCor)),len(X))

./cluster/clust_0.csv B2CL1 0.02 0.10 4983
./cluster/clust_1.csv B2CL1 0.30 0.10 345
./cluster/clust_10.csv B2CL1 0.15 0.10 330
./cluster/clust_11.csv B2CL1 0.04 0.10 320
./cluster/clust_12.csv B2CL1 0.00 0.11 278
./cluster/clust_2.csv B2CL1 0.09 0.09 787
./cluster/clust_3.csv B2CL1 0.28 0.10 804
./cluster/clust_4.csv B2CL1 0.03 0.09 309
./cluster/clust_5.csv B2CL1 0.02 0.10 377
./cluster/clust_6.csv B2CL1 0.10 0.11 348
./cluster/clust_7.csv B2CL1 0.05 0.09 433
./cluster/clust_8.csv B2CL1 0.18 0.12 455
./cluster/clust_9.csv B2CL1 0.13 0.08 197
./cluster/clust_0.csv MCL1 0.32 0.14 4491
./cluster/clust_1.csv MCL1 0.18 0.08 334
./cluster/clust_10.csv MCL1 0.22 0.10 236
./cluster/clust_11.csv MCL1 0.08 0.14 278
./cluster/clust_12.csv MCL1 0.07 0.11 256
./cluster/clust_2.csv MCL1 0.40 0.07 703
./cluster/clust_3.csv MCL1 0.41 0.08 730
./cluster/clust_4.csv MCL1 0.10 0.14 271
./cluster/clust_5.csv MCL1 0.05 0.08 382
./cluster/clust_6.csv MCL1 0.30 0.11 244
./cluster/clust_7.csv MCL1 0.09 0.13

In [16]:
model_csv = "./results/"+model+"_cor.csv"
pd.DataFrame(AllCors).to_csv(model_csv )

In [17]:
#This file is big, write somewhere with lots of space
model_pickle = "/scratch/users/vfrap/BayesianModel/"+model+".pk"
pickle.dump(trace,open(model_pickle,"wb"))




FileNotFoundError: [Errno 2] No such file or directory: '/scratch/users/vfrap/BayesianModel/Test_9_MCL1_rand_0_poly_0_Prior_No_PrioStr_1.000.pk'